<a href="https://colab.research.google.com/github/jki5410/DE-3rd/blob/main/Spark/PySpark_%EC%84%A4%EC%B9%98_%EB%B0%8F_%ED%85%8C%EC%8A%A4%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PySpark을 로컬머신에 설치하고 노트북을 사용하기 보다는 머신러닝 관련 다양한 라이브러리가 이미 설치되었고 좋은 하드웨어를 제공해주는 Google Colab을 통해 실습을 진행한다. 이를 위해 pyspark과 Py4J 패키지를 설치한다.

PySpark을 설치하면 Local Standalone Spark이 설치되고 이 Spark 클러스터를 파이썬에서 액세스해야되기 때문에 Py4J 설치가 필요하다.
Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다.

# PySpark 설치

In [1]:
!pip install pyspark==3.3.1 py4j==0.10.9.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 18.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845493 sha256=fc6743773ea7f5aaa212a1ed27ca0ae43fc9e848bf63adce7765d96d3c21a5ee
  Stored in directory: /root/.cache/pip/wheels/0f/f0/3d/517368b8ce80486e84f89f214e0a022554e4ee64969f46279b
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


**Spark Session:** SparkSession은 Spark 2.0부터 엔트리 포인트로 사용된다. SparkSession을 이용해 RDD, 데이터 프레임등을 만든다. SparkSession은 SparkSession.builder를 호출하여 생성하며 다양한 함수들을 통해 세부 설정이 가능하다

* local[*] Spark이 하나의 JVM으로 동작하고 그 안에 컴퓨터의 코어 수 만큼의 스레드가 Executor로 동작한다


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark Tutorial')\
        .getOrCreate()

In [3]:
spark

In [4]:
!lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   2
  On-line CPU(s) list:    0,1
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:           6
    Model:                79
    Thread(s) per core:   2
    Core(s) per socket:   1
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4400.36
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 cl
                          flush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc re
                          p_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3
                           fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand
                           hypervisor lahf_lm abm 3dnowprefetch i

cpu가 두 개!
즉, 두 개의 쓰레드가 돌 수 있다.

In [5]:
!grep MemTotal /proc/meminfo

MemTotal:       13290464 kB


# Python <> RDD <> DataFrame

### **Python 객체를 RDD로 변환해보기**

**1> Python 리스트 생성**

In [6]:
name_list_json = [ '{"name": "keeyong"}', '{"name": "benjamin"}', '{"name": "claire"}' ]

json처럼 보이지만, 작은 따옴표로 구성되어있는 string 인스턴스로 나열된 리스트이다

In [7]:
for n in name_list_json:
  print(n)

{"name": "keeyong"}
{"name": "benjamin"}
{"name": "claire"}


**2> 파이썬 리스트를 RDD로 변환**

 * RDD로 변환되는 순간 Spark 클러스터의 서버들에 데이터가 나눠 저장됨 (파티션)

위에서 spark session object를 만들어 spark이라는 변수에 저장했던 해당 spark변수를 활용하여 RDD로 변환하는 과정!

sparkContext를 이용해 RDD와 관련된 작업들을 할 수 있다.

parallelize함수를 부르며 위의 스트링으로 구성된 파이썬 리스트를 넘기면 RDD가 만들어진다.


In [9]:
rdd = spark.sparkContext.parallelize(name_list_json)

In [12]:
rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

Spark이 lazy execution을 하기때문에 이 시점에는 아무것도 하지 않다가 실제 데이터가 계산이 되어야만 가능한 작업들을 하는 순간 그때서야 미뤄놨던 작업들을 실행한다.

(spark 클러스터로 올리는 작업을 ..!)

In [11]:
rdd.count()

3

RDD의 장점?! -> lambda function을 쓰기 좋다..?

람다 함수를 써서 string을 json형식으로 (dictionary형태로) 바꾸기  

** immutable object이기때문에 생성했던 RDD의 값을 바꿀수는 없고 새로운 RDD 생성!

In [13]:
import json

parsed_rdd = rdd.map(lambda el:json.loads(el))

In [14]:
parsed_rdd

PythonRDD[2] at RDD at PythonRDD.scala:53

collect함수를 이용하여 RDD를 Python쪽으로 가져오기

In [15]:
parsed_rdd.collect()

[{'name': 'keeyong'}, {'name': 'benjamin'}, {'name': 'claire'}]

In [16]:
parsed_name_rdd = rdd.map(lambda el:json.loads(el)["name"])

In [17]:
parsed_name_rdd.collect()

['keeyong', 'benjamin', 'claire']

###**파이썬 리스트를 데이터프레임으로 변환하기**

파이썬 리스트나 판다스 데이터프레임이나 RDD를 Spark 데이터 프레임으로 변환하고 싶은 경우에는 Spark 세션에 있는 createDataFrame함수를 이용해주면 된다.

createDataFrame함수에서 두 번째 인자로 스키마를 지정해줘야하는데, 파이썬 리스트같은 경우 string 타입이 리스트이므로 일단은 StringType()으로 지정!..
이 경우 디폴트로 필드의 이름을 value라고 지정해준다

In [18]:
from pyspark.sql.types import StringType

df = spark.createDataFrame(name_list_json, StringType())

In [19]:
df.count()

3

In [20]:
df.printSchema()

root
 |-- value: string (nullable = true)



ㄴvalue라는 필드가 있고 해당 필드의 데이터 타입이 string이다

df의 모든 데이터를 모아서 collect로 파이썬으로 가져오기..

In [21]:
df.select('*').collect()

[Row(value='{"name": "keeyong"}'),
 Row(value='{"name": "benjamin"}'),
 Row(value='{"name": "claire"}')]

ㄴRow라는 타입으로 표시되어있는데, 데이터셋에서 데이터프레임을 지원하기 위해 사용되는 타입..

일단은 데이터프레임을 파이썬쪽으로 읽어오면 레코드들이 Row타입으로 표시된다고 이해하면 될 것임 ...

RDD를 DataFrame으로 변환해보는 예제: 앞서 parsed_rdd를 DataFrame으로 변환해보자



In [22]:
df_parsed_rdd = parsed_rdd.toDF()

In [23]:
df_parsed_rdd.printSchema()

root
 |-- name: string (nullable = true)



In [24]:
df_parsed_rdd.select('name').collect()

[Row(name='keeyong'), Row(name='benjamin'), Row(name='claire')]

# Spark 데이터프레임으로 로드해보기

In [25]:
!wget https://s3-geospatial.s3-us-west-2.amazonaws.com/name_gender.csv

--2024-06-19 08:22:21--  https://s3-geospatial.s3-us-west-2.amazonaws.com/name_gender.csv
Resolving s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)... 52.92.192.18, 52.218.197.89, 52.218.181.49, ...
Connecting to s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)|52.92.192.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 997 [text/csv]
Saving to: ‘name_gender.csv’

name_gender.csv     100%[===================>]     997  --.-KB/s    in 0s      

2024-06-19 08:22:21 (17.1 MB/s) - ‘name_gender.csv’ saved [997/997]



In [26]:
df = spark.read.csv("name_gender.csv")
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



header가 있음을 알리고 스키마가 제대로 잡히게 해보자!..

In [27]:
df = spark.read.option("header", True).csv("name_gender.csv")
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)



In [28]:
df.show()

+----------+------+
|      name|gender|
+----------+------+
|  Adaleigh|     F|
|     Amryn|Unisex|
|    Apurva|Unisex|
|    Aryion|     M|
|    Alixia|     F|
|Alyssarose|     F|
|    Arvell|     M|
|     Aibel|     M|
|   Atiyyah|     F|
|     Adlie|     F|
|    Anyely|     F|
|    Aamoni|     F|
|     Ahman|     M|
|    Arlane|     F|
|   Armoney|     F|
|   Atzhiry|     F|
| Antonette|     F|
|   Akeelah|     F|
| Abdikadir|     M|
|    Arinze|     M|
+----------+------+
only showing top 20 rows



In [29]:
df.head(5)

[Row(name='Adaleigh', gender='F'),
 Row(name='Amryn', gender='Unisex'),
 Row(name='Apurva', gender='Unisex'),
 Row(name='Aryion', gender='M'),
 Row(name='Alixia', gender='F')]

In [30]:
df.groupby(["gender"]).count().collect()

[Row(gender='F', count=65),
 Row(gender='M', count=28),
 Row(gender='Unisex', count=7)]

ㄴdf.groupby(["gender"]).count()만 쓰면 Spark 클러스터에서만 동작!
collect를 호출하여 python 드라이버로 가져와서 내용을 확인하기!

In [31]:
df.rdd.getNumPartitions()

1

### 데이터프레임을 테이블뷰로 만들어서 SparkSQL로 처리해보기

SQL을 활용하면... 굉장히 편리하지요? 간단하고!?

In [32]:
df.createOrReplaceTempView("namegender")

In [33]:
namegender_group_df = spark.sql("SELECT gender, count(1) FROM namegender GROUP BY 1")

In [34]:
namegender_group_df.collect()

[Row(gender='F', count(1)=65),
 Row(gender='M', count(1)=28),
 Row(gender='Unisex', count(1)=7)]

어떤 데이터 프레임들이 테이블로 사용가능한지 알고싶으면 -> catalog.listTables

In [35]:
spark.catalog.listTables()

[Table(name='namegender', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

Partition의 수 계산해보기

In [36]:
namegender_group_df.rdd.getNumPartitions()

1

파티션을 두 개로 나누기?

In [37]:
two_namegender_group_df = namegender_group_df.repartition(2)

In [38]:
two_namegender_group_df.rdd.getNumPartitions()

2